In [1]:
!pip install huggingface_hub
!pip install openai langchain langchain-openai tiktoken python-dotenv rich transformers torch
!pip install langchain-community
!pip install faiss-cpu
!pip install transformers==4.47.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433

In [19]:
import os
os.environ["GROQ_API_KEY"] = "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"

In [20]:
from __future__ import annotations
import os
import requests
from typing import List, Dict
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.documents import Document
from openai import OpenAI
class LawyerAgent:
    def __init__(self, name: str, system_prompt: str, model: str = "llama-3.3-70b-versatile"):
        self.name = name
        self.system_prompt = system_prompt.strip()
        self.history: List[Dict[str, str]] = []
        self.model = model
        self.api_key = os.getenv("GROQ_API_KEY")

        self.client = OpenAI(
            api_key=self.api_key,
            base_url="https://api.groq.com/openai/v1"
        )

    def _format_prompt(self, user_msg: str) -> List[Dict[str, str]]:
        messages = [{"role": "system", "content": self.system_prompt}]
        messages.extend(self.history)
        messages.append({"role": "user", "content": user_msg})
        return messages

    def respond(self, user_msg: str) -> str:
        messages = self._format_prompt(user_msg)

        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            temperature=0.7,
            max_tokens=300
        )

        reply = response.choices[0].message.content

        self.history.append({"role": "user", "content": user_msg})
        self.history.append({"role": "assistant", "content": reply})

        return reply

In [36]:
import pandas as pd

# Load the CSV or other data source
df = pd.read_csv('/content/cases.csv')  # Replace with the actual filename

# Optional: Drop the unnamed index column if needed
df = df.drop(columns=['Unnamed: 0'], errors='ignore')

# Iterate through each row
for index, row in df.iterrows():
    doc_id = row['id']
    text = row['text']
    print(text)


civil appellate jurisdiction civil appeal number 1709
of
1988.
from the judgment and order dated 9.10.1987 of t
he
central administrative tribunal chandigarh in appln. t.
--
1055 of 1986.
with
 slp  civil  number 6998 of 1988 

c. mahajan mrs. indu goswami c.v. subba rao
p.
parmeshwaran for the appellant in c.a. number 1709 of 1988.

k.d. namboodary for the petitioner in slp  civil  n
o. 6998 of 1988.

m. ashri and mahabir singh for the respondents. the judgment of the companyrt was delivered by

jagannatha shetty j. the civil appeal by speci
al
leave and the companynected slp raise an important issue as
to
the power of the central administration tribun
al
 tribunal  to examine the adequacy of penalty awarded
by
the companypetent authority to a government servant in discipl
i-
nary proceedings. short factual background is this
parmanand--respondent in the appeal was a time keeper
in
beas sutlej link project sundernagar. he was incharge
of
preparing the pay bills and other bills of the 

In [12]:
JUDGE_SYSTEM = """
You are Judge. Decide according to indian constitution,ipc,crpc and cpc. You oversee courtroom proceedings, responding formally and only when appropriate. You may cross-question speakers (except Police), call witnesses, or say “Pass” when no response is needed. You must reply when addressed with “My Lord.” Give judgement after you have listened to everyone. Output GRANTED if accused is convict otherwise DENIED”
"""

DEFENSE_SYSTEM = """
You are  the defense attorney.
You respond to courtroom dialogue, address the judge as "My Lord," and may speak for your client or question others when appropriate.
Only question witnesses when allowed by the judge.
Stay professional, factual, and respectful.
Say "Pass" if you choose not to respond.
After judgment, thank the judge.
"""

PROSECUTION_SYSTEM = """
You are  the prosecutor.
You respond to the judge, defendant, or witnesses, and may present evidence or request actions.
Only question witnesses when allowed by the judge.
Stay formal, ethical, and persuasive.
Say "Pass" if you choose not to respond.
After judgment, thank the judge.
"""

ACCUSED_SYSTEM = """
You are the Accused.
Speak only when asked by the Judge, Prosecutor, or Defense Attorney — otherwise say "Pass."
Be truthful, respectful, and emotionally sincere.
Do not argue law or make up evidence.
"""

ACCUSER_SYSTEM = """
You are the Accuser.
Speak only when addressed, otherwise say "Pass."
Be truthful, serious, and respectful.
Speak only from personal experience — no lies or exaggeration.
Begin with "My Lord" when addressing the Judge.
"""

DIRECTOR_SYSTEM = """
You are the courtroom simulation director.
You can add realistic evidence or witnesses to build suspense.
Use the correct format for dialogue, evidence, and witness info.
Say "Pass" if you choose to take no action.
Start moving the case toward closure after 6–7 rounds.
Keep the tone narrative-driven but realistic.
"""


judge = LawyerAgent("Judge", JUDGE_SYSTEM)
prosecution = LawyerAgent("Prosecution", PROSECUTION_SYSTEM)
defense = LawyerAgent("Defense", DEFENSE_SYSTEM)
accused = LawyerAgent("Accused", ACCUSED_SYSTEM)
accuser = LawyerAgent("Accuser", ACCUSER_SYSTEM)
director = LawyerAgent("Director", DIRECTOR_SYSTEM)



In [11]:
import pandas as pd

# Load the CSV or other data source
df = pd.read_csv('/content/cases.csv')  # Replace with the actual filename

# Optional: Drop the unnamed index column if needed
df = df.drop(columns=['Unnamed: 0'], errors='ignore')
flag=True
data=[]
# Iterate through each row
for index, row in df.iterrows():
    if index <=51:
      judge=LawyerAgent('judge','''
  You are a judge. You are given a summary of case.You have to decide on basis of the Indian Constitution, IPC, CPC, CRPC .You have to respond with 'GRANTED' or 'DENIED'. If the accused is convicted respond with 'GRANTED' and if not respond with 'DENIED'.
''')
      doc_id = row['id']
      text = row['text'][:2000]
      response=judge.respond(text)
      print(response)
      if 'GRANTED.' in response:
        data.append({'id':doc_id,'verdict':1})
      else:
        data.append({'id':doc_id,'verdict':0})
    else:
      pass
print(data)
df = pd.DataFrame(data)
df.to_csv("/content/verdicts.csv", index=False)


GRANTED. 

Reasoning: The case involves a government servant, Parmanand, who was found to have masterminded a scheme to withdraw the pay of a non-working employee, Ashok Kumar, by preparing a fake pay roll and identity card. The respondent's actions constitute a serious breach of trust and misconduct, as outlined in the Indian Constitution, specifically Article 311, which deals with the dismissal, removal, or reduction in rank of a person employed in a civil capacity under the Union or a State. 

Additionally, the respondent's actions can be considered as an offense under the Indian Penal Code (IPC), specifically Section 420, which deals with cheating and dishonestly inducing delivery of property. 

The Central Administrative Tribunal (CAT) has the power to examine the adequacy of the penalty awarded by the competent authority, as per the provisions of the Administrative Tribunals Act, 1985. 

Given the seriousness of the offense and the respondent's misconduct, it is likely that the c

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01js7krnjae0ht0w3x6hmtzrmj` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99939, Requested 573. Please try again in 7m22.229999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [44]:
df.to_csv("output.csv", index=False)

In [13]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
documents = [Document(page_content='It is better to forgo 1000 evil doers to punishing an innocent')]
vector_space = FAISS.from_documents(documents, embeddings)
retriever = vector_space.as_retriever()

<ipython-input-13-5e4be204d926>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import pandas as pd

# Load the CSV or other data source
df = pd.read_csv('/content/cases.csv')  # Replace with the actual filename

# Optional: Drop the unnamed index column if needed
df = df.drop(columns=['Unnamed: 0'], errors='ignore')
flag=True
data=[]
# Iterate through each row
for index, row in df.iterrows():
    if index <=51:
      JUDGE_SYSTEM = """
You are Judge. Decide according to indian constitution,ipc,crpc and cpc.Keep the trial short .You oversee courtroom proceedings, responding formally and only when appropriate. You may cross-question speakers (except Police), call witnesses, or say “Pass” when no response is needed. You must reply when addressed with “My Lord.” Give judgement after you have listened to everyone. Output GRANTED if accused is convict otherwise DENIED.Pronounce the judgement after two witnesses have been called. Please generate GRANTED or DENIED as soon as you understand”
"""

      DEFENSE_SYSTEM = """
      In each subsmission produce no more than 50 words
      You are  the defense attorney.
      You respond to courtroom dialogue, address the judge as "My Lord," and may speak for your client or question others when appropriate.
      Only question witnesses when allowed by the judge.
      Stay professional, factual, and respectful.
      Say "Pass" if you choose not to respond.
      After judgment, thank the judge.
      """

      PROSECUTION_SYSTEM = """
      In each subsmission produce no more than 50 words
      You are  the prosecutor.
      You respond to the judge, defendant, or witnesses, and may present evidence or request actions.
      Only question witnesses when allowed by the judge.
      Stay formal, ethical, and persuasive.
      Say "Pass" if you choose not to respond.
      After judgment, thank the judge.
      """

      ACCUSED_SYSTEM = """
      In each subsmission produce no more than 50 words
      You are the Accused.
      Speak only when asked by the Judge, Prosecutor, or Defense Attorney — otherwise say "Pass."
      Be truthful, respectful, and emotionally sincere.
      Do not argue law or make up evidence.
      """

      ACCUSER_SYSTEM = """
      In each subsmission produce no more than 50 words
      You are the Accuser.
      Speak only when addressed, otherwise say "Pass."
      Be truthful, serious, and respectful.
      Speak only from personal experience — no lies or exaggeration.
      Begin with "My Lord" when addressing the Judge.
      """

      DIRECTOR_SYSTEM = """
      In each subsmission produce no more than 50 words
      There is limit of 200 words for witness' submission
      You are the courtroom simulation director.
      You can add realistic evidence or witnesses to build suspense.
      Use the correct format for dialogue, evidence, and witness info.
      Say "Pass" if you choose to take no action.
      Start moving the case toward closure after 6–7 rounds.
      Keep the tone narrative-driven but realistic.
      """


      judge = LawyerAgent("Judge", JUDGE_SYSTEM)
      prosecution = LawyerAgent("Prosecution", PROSECUTION_SYSTEM)
      defense = LawyerAgent("Defense", DEFENSE_SYSTEM)
      accused = LawyerAgent("Accused", ACCUSED_SYSTEM)
      accuser = LawyerAgent("Accuser", ACCUSER_SYSTEM)
      director = LawyerAgent("Director", DIRECTOR_SYSTEM)

      doc_id = row['id']
      text = row['text']
      conersation_response=''
      # ==== Run the courtroom simulation ====

      N = 1
      history = []
      context = text

      announce = "Judge: Let the trial begin. I call upon the prosecutor to give opening remarks."

      while True:
          context_docs = retriever.get_relevant_documents(announce)
          context = "\n".join([doc.page_content for doc in context_docs])
          prompt = f"N: {N}\nContext: {context}\nImmediate proceeding in court: {announce}"

          judge_resp = judge.respond(prompt)
          prosecution_resp = prosecution.respond(prompt)
          defense_resp = defense.respond(prompt)
          accused_resp = accused.respond(prompt)
          accuser_resp = accuser.respond(prompt)
          director_resp = director.respond(prompt)

          # if judge_resp != "Pass":
          #     print("JUDGE:", judge_resp)
          # if prosecution_resp != "Pass":
          #     print("PROSECUTOR:", prosecution_resp)
          # if defense_resp != "Pass":
          #     print("DEFENSE:", defense_resp)
          # if accused_resp != "Pass":
          #     print("ACCUSED:", accused_resp)
          # if accuser_resp != "Pass":
          #     print("ACCUSER:", accuser_resp)
          # if director_resp != "Pass":
          #     print("DIRECTOR:", director_resp)

          announce = ""
          if judge_resp != "Pass":
              announce += f"Judge: {judge_resp}\n"
          if prosecution_resp != "Pass":
              announce += f"Prosecutor: {prosecution_resp}\n"
          if defense_resp != "Pass":
              announce += f"Defense: {defense_resp}\n"
          if accused_resp != "Pass":
              announce += f"Accused: {accused_resp}\n"
          if accuser_resp != "Pass":
              announce += f"Accuser: {accuser_resp}\n"
          if director_resp != "Pass":
              announce += f"{director_resp}\n"

          if "GRANTED" in judge_resp:
              conversation_response='GRANTED'
              break
          elif "DENIED" in judge_resp:
              conversation_response='DENIED'
              break

          N += 1
          if N > 100:
              print("Ending simulation: too many rounds without closure.")
              break
      response=conversation_response
      print(response)
      if 'GRANTED' in response:
        data.append({'id':doc_id,'verdict':1})
      else:
        data.append({'id':doc_id,'verdict':0})
    else:
      pass
print(data)
df = pd.DataFrame(data)
df.to_csv("/content/verdicts.csv", index=False)

GRANTED
GRANTED
